<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
import os
import sys
import glob
import json
import datetime as dt
from pathlib import Path
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from dotenv import load_dotenv

In [11]:
# Carregar variáveis do arquivo .env
load_dotenv()


True

In [12]:
def get_mt5_credentials():
    """Carrega credenciais do MT5 do arquivo .env"""
    return {
        'login': int(os.getenv('MT5_LOGIN', 0)),
        'password': os.getenv('MT5_PASSWORD', ''),
        'server': os.getenv('MT5_SERVER', ''),
        'path': os.getenv('MT5_PATH', r"C:\Program Files\MetaTrader 5\terminal64.exe")
    }


def load_config(config_file):
    """Carrega todas as configurações do arquivo JSON"""
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Arquivo {config_file} não encontrado!")
        return None
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON do arquivo {config_file}")
        return None
    except Exception as e:
        print(f"Erro ao carregar {config_file}: {e}")
        return None


def connect_mt5(config=None):
    """Estabelece conexão com MT5"""
    if config is None:
        config = get_mt5_credentials()
    
    # Validar credenciais
    if not config['login'] or not config['password'] or not config['server']:
        print("Erro: Credenciais do MT5 não encontradas no arquivo .env")
        print("Certifique-se de que MT5_LOGIN, MT5_PASSWORD e MT5_SERVER estão definidos")
        return False
    
    if not mt5.initialize(
        login=config['login'], 
        server=config['server'], 
        password=config['password'], 
        path=config['path']
    ):
        print("initialize() failed, error code =", mt5.last_error())
        return False
    
    print('Ligado ao MT5 com sucesso!')
    print(f'Conta: {config["login"]} | Servidor: {config["server"]}')
    print('-' * 20)
    return True

In [13]:
connect_mt5()

Ligado ao MT5 com sucesso!
Conta: 3008705129 | Servidor: Rico-DEMO
--------------------


True

In [17]:
def get_timeframe_offset(timeframe):
    """Retorna o offset em minutos baseado no timeframe"""
    timeframe_map = {
        't1': 1, 't2': 2, 't5': 5, 't10': 10, 't15': 15, 't30': 30,
        'h1': 60, 'h4': 240, 'd1': 1440,
    }
    return timeframe_map.get(timeframe.lower(), 5)


def trade_report(symbol, data_ini, data_fim, cost_per_lot=0.25):
    """Extrai relatório de trades do MT5"""
    deals = mt5.history_deals_get(data_ini, data_fim, group=symbol)
    
    if deals is None or len(deals) == 0:
        print(f"Nenhum deal encontrado para {symbol} no período especificado")
        return pd.DataFrame()
    
    print(f"Encontrados {len(deals)} deals para {symbol}")
    
    df = pd.DataFrame(list(deals), columns=deals[0]._asdict().keys())
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['custo'] = df['volume'] * cost_per_lot
    df['lucro'] = df['profit'] - df['custo']
    
    return df




In [21]:
data_ini = '2025-06-01'
data_fim = '2025-06-27'
symbol = '*WIN*'
cost_per_lot = 0.5

# Extrair dados do MT5
dfmt5 = trade_report(
    symbol, 
    pd.Timestamp(data_ini), 
    pd.Timestamp(data_fim ) + dt.timedelta(days=1),
    cost_per_lot
)

Encontrados 496 deals para *WIN*


In [26]:
dfmt5[dfmt5['position_id']==2134095577]

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,custo,lucro
485,1575014545,2134095577,2025-06-27 12:50:00,1751028600286,0,0,9783,2134095577,3,1.0,139350.0,0.0,0.0,0.0,0.0,WINQ25,bb_trend,,0.5,-0.5
493,1575527543,2134808398,2025-06-27 17:54:00,1751046840028,1,1,0,2134095577,3,1.0,139140.0,0.0,0.0,-42.0,0.0,WINQ25,,,0.5,-42.5


In [27]:
dfmt5['comment'].value_counts()

comment
pattern_rsi_tren    205
                    158
bb_trend             29
gold_rsi_trend       13
[sl 138905]           2
                   ... 
[sl 141815]           1
[sl 141795]           1
[sl 141780]           1
[sl 141765]           1
[sl 139185]           1
Name: count, Length: 89, dtype: int64

In [24]:
dfmt5['magic'].value_counts()

magic
52455    241
0        158
1111      32
9783      29
3598      19
2         17
Name: count, dtype: int64

In [29]:
dfmt5[dfmt5.magic.isnull()]

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,custo,lucro
